In [1]:
from langchain import FAISS
from langchain.document_loaders import TextLoader
from langchain.embeddings import HuggingFaceEmbeddings, OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter

# 加载文本
story = TextLoader('data/story.txt').load()
# transform定义
text_splitter = CharacterTextSplitter(
    separator = "\n\n",
    chunk_size = 100,
    chunk_overlap  = 20,
    length_function = len,
)
# transform出来
texts = text_splitter.split_documents(story)
print(len(texts))
# 加载到vector store
db = FAISS.from_documents(documents=texts,embedding=OpenAIEmbeddings())

Created a chunk of size 141, which is longer than the specified 100
Created a chunk of size 121, which is longer than the specified 100
Created a chunk of size 130, which is longer than the specified 100


4


In [2]:
# 指定取几个参数
retriever = db.as_retriever(search_kwargs={"k": 1})

In [3]:
docs = retriever.get_relevant_documents("托马斯是一名时间旅行者？")
print(docs)

[Document(page_content='故事三：时间旅行者的恋情\n托马斯是一名时间旅行者，他在不同的时代中穿梭。在一次时间旅行中，他在18世纪遇到了美丽的女子艾米丽。托马斯深深地爱上了艾米丽，但他们因时代的差异而不能在一起。在经历了一系列的冒险和挑战后，托马斯最终决定留在18世纪，与艾米丽共度一生。', metadata={'source': 'data/story.txt'})]


In [5]:
# 检索类型，默认情况下，向量存储检索器使用相似度搜索。如果底层向量存储支持最大边际相关性搜索，则可以将其指定为搜索类型。
# 最大边际相关性检索的主要思想是在选择结果时，不仅要考虑结果与查询的相关性，还要考虑结果之间的差异性。也就是说，它试图在相关性和多样性之间找到一个平衡，以提供更有价值的信息。
# 具体来说，最大边际相关性检索会首先选择与查询最相关的结果，然后在后续的选择中，会优先选择与已选择结果差异较大的结果。这样，返回的结果既能覆盖查询的主要相关信息，又能提供多样的视角和内容，从而减少冗余。
retriever = db.as_retriever(search_type="mmr",search_kwargs={"k": 1})
docs = retriever.get_relevant_documents("托马斯是一名时间旅行者？")
print(docs)

[Document(page_content='故事三：时间旅行者的恋情\n托马斯是一名时间旅行者，他在不同的时代中穿梭。在一次时间旅行中，他在18世纪遇到了美丽的女子艾米丽。托马斯深深地爱上了艾米丽，但他们因时代的差异而不能在一起。在经历了一系列的冒险和挑战后，托马斯最终决定留在18世纪，与艾米丽共度一生。', metadata={'source': 'data/story.txt'})]


In [9]:
retriever = db.as_retriever(search_type="similarityatscore_threshold", search_kwargs={"score_threshold": .5})
docs = retriever.get_relevant_documents("托马斯是一名时间旅行者？")

ValueError: search_type of similarityatscore_threshold not allowed.